## 0.0. Import des librairies

In [2]:
import pandas as pd
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython import display

## 0.1. Mot à rechercher

In [3]:
word = input("Quel mot rechercher ? ")

Quel mot rechercher ? abacus


## 1.1. Requeter Linguee

In [4]:
urlTarget = f"https://www.linguee.fr/anglais-francais/traduction/{word}.html"
response = requests.get(urlTarget)
love = BeautifulSoup(response.text, 'html.parser')
body = BeautifulSoup(response.text, 'html.parser').find('body')

## 1.2. Extraire les audio US et UK du mot

In [5]:
span = body.find('span',{"class":"tag_lemma"})
pos = span.findAll('span',{"class":"tag_wordtype"})[0].text
audio_US = span.find('a',{"class":"audio"})['onclick'].split('"')[1]
audio_UK = span.find('a',{"class":"audio"})['onclick'].split('"')[5]
audio_US_url = f"https://www.linguee.fr/mp3/{audio_US}.mp3"
audio_UK_url = f"https://www.linguee.fr/mp3/{audio_UK}.mp3"

print("Mot :", word.capitalize(), "\nPOS :", pos)
print("Prononciation US :", audio_US_url)
display.display(display.Audio(url=audio_US_url))
print("Prononciation UK :", audio_UK_url)
display.display(display.Audio(url=audio_UK_url))

Mot : Abacus 
POS : nom
Prononciation US : https://www.linguee.fr/mp3/EN_US/13/13f27b1072bbf7719d0d267b083ff91c-101.mp3


Prononciation UK : https://www.linguee.fr/mp3/EN_UK/13/13f27b1072bbf7719d0d267b083ff91c-101.mp3


## 1.3. Extraire les audio US et UK de tous les POS du mot

In [6]:
span = body.findAll('span',{"class":"tag_lemma"})
df = pd.DataFrame(columns=["Mot", "IPA", "POS", "Link_US", "Link_UK",])
for i,v in enumerate(span):
    name = v.findAll('a',{"class":"dictLink"})
    # l'IPA n'est pas mentionné dans Linguee
    ipa = ""
    link = v.findAll('a',{"class":"audio"})
    pos = v.findAll('span',{"class":"tag_wordtype"})
    if len(pos)>0:
        for l in link:
            audio_US = l['onclick'].split('"')[1]
            audio_UK = l['onclick'].split('"')[5]
            audio_US_url = f"https://www.linguee.fr/mp3/{audio_US}.mp3"
            audio_UK_url = f"https://www.linguee.fr/mp3/{audio_UK}.mp3"
            df.loc[i] = [name[0].text, ipa, pos[0].text, audio_US_url, audio_UK_url]
df

,Mot,IPA,POS,Link_US,Link_UK
0,abacus,,nom,https://www.linguee.fr/mp3/EN_US/13/13f27b1072...,https://www.linguee.fr/mp3/EN_UK/13/13f27b1072...


In [7]:
print("Mot :", df.Mot[0].capitalize())

out = widgets.Output()
with out:
    tableau_label = '<table><tr><th style="text-align:center">IPA</th>\
                                <th style="text-align:center">POS</th>\
                                <th style="text-align:center">Audio US</th>\
                                <th style="text-align:center">Audio UK</th>\
                                </tr>'
    for i in range(len(df)):
        
        
        tableau_label += '<tr><td>'+df.IPA[i]+'</td>\
                            <td>'+df.POS[i]+'</td>\
                              <td> <audio controls src="'+df.Link_US[i]+'"><code>audio</code></audio></td>\
                              <td> <audio controls src="'+df.Link_UK[i]+'"><code>audio</code></audio></td>\
                          </tr>'


    tableau_label += '</table>'
    tableau = display.HTML(tableau_label)
    display.display(tableau)
        

display.display(out)


Mot : Abacus


Output()

## 2.1. Requeter Cambridge

In [8]:
session = HTMLSession()
urlTarget = f"https://dictionary.cambridge.org/dictionary/english/{word}"
response = session.get(urlTarget)
love = BeautifulSoup(response.text, 'html.parser')
body = BeautifulSoup(response.text, 'html.parser').find('body')

## 2.2. Extraire l'IPA et les audio US et UK du mot

In [9]:
mots = body.find('div',{"class":"pos-header dpos-h"})
pos = mots.findAll('div',{"class":"posgram dpos-g hdib lmr-5"})[0].text.split(" ")[0]
ipa = mots.findAll('span',{"class":"pron dpron"})[0].text.replace("/", "")
audio_UK = mots.findAll('span',{"class":"uk dpron-i"})[0].findAll('source')[0]["src"]
audio_US = mots.findAll('span',{"class":"us dpron-i"})[0].findAll('source')[0]["src"]
audio_US_url = f"https://dictionary.cambridge.org{audio_US}"
audio_UK_url = f"https://dictionary.cambridge.org{audio_UK}"

print("Mot :", word.capitalize(), "\nIPA :", ipa, "\nPOS :", pos)
print("Prononciation US :", audio_US_url)
display.display(display.Audio(url=audio_US_url))
print("Prononciation UK :", audio_UK_url)
display.display(display.Audio(url=audio_UK_url))

Mot : Abacus 
IPA : ˈæb.ə.kəs 
POS : noun
Prononciation US : https://dictionary.cambridge.org/media/english/us_pron/a/aba/abacu/abacus.mp3


Prononciation UK : https://dictionary.cambridge.org/media/english/uk_pron/u/uka/uka__/uka____012.mp3


## 2.3. Extraire l'IPA et les audio US et UK de tous les POS du mot

In [10]:
mots = body.findAll('div',{"class":"pos-header dpos-h"})
df = pd.DataFrame(columns=["Mot", "IPA", "POS", "Link_US", "Link_UK",])
for i,m in enumerate(mots):
    name = m.findAll('span',{"class":"hw dhw"})[0].text
    ipa = m.findAll('span',{"class":"pron dpron"})[0].text.replace("/", "")
    div = m.findAll('div',{"class":"posgram dpos-g hdib lmr-5"})
    for d in div:
        pos = d.text.split(" ")[0]
        try:
            audio_UK = m.findAll('span',{"class":"uk dpron-i"})[0].findAll('source')[0]["src"]
            audio_US = m.findAll('span',{"class":"us dpron-i"})[0].findAll('source')[0]["src"]
            audio_US_url = f"https://dictionary.cambridge.org{audio_US}"
            audio_UK_url = f"https://dictionary.cambridge.org{audio_UK}"
            df.loc[i] = [name, ipa, pos, audio_US_url, audio_UK_url]
        except:
            ok = 1
df

,Mot,IPA,POS,Link_US,Link_UK
0,abacus,ˈæb.ə.kəs,noun,https://dictionary.cambridge.org/media/english...,https://dictionary.cambridge.org/media/english...


## 3.1. Requeter Longman

In [11]:
session = HTMLSession()
urlTarget = f"https://www.ldoceonline.com/dictionary/{word}"
response = session.get(urlTarget)
love = BeautifulSoup(response.text, 'html.parser')
body = BeautifulSoup(response.text, 'html.parser').find('body')

## 3.2. Extraire l'IPA et les audio US et UK du mot

In [12]:
mots = body.find('span',{"class":"Head"})
pos = mots.findAll('span',{"class":"POS"})[0].text
ipa = mots.findAll('span',{"class":"PRON"})[0].text
audio_UK_url = mots.findAll('span',{"class":"speaker brefile fas fa-volume-up hideOnAmp"})[0]["data-src-mp3"]
audio_US_url = mots.findAll('span',{"class":"speaker amefile fas fa-volume-up hideOnAmp"})[0]["data-src-mp3"]

print("Mot :", word.capitalize(), "\nIPA :", ipa, "\nPOS :", pos)
print("Prononciation US :", audio_US_url)
display.display(display.Audio(url=audio_US_url.split("?")[0]))
print("Prononciation UK :", audio_UK_url)
display.display(display.Audio(url=audio_UK_url.split("?")[0]))

Mot : Abacus 
IPA : ˈæbəkəs 
POS :  noun
Prononciation US : https://www.ldoceonline.com/media/english/ameProns/abacus.mp3?version=1.2.42


Prononciation UK : https://www.ldoceonline.com/media/english/breProns/abacus0205.mp3?version=1.2.42


## 3.3. Extraire les audio US et UK de tous les POS du mot

In [13]:
mots = body.findAll('span',{"class":"Head"})
df = pd.DataFrame(columns=["Mot", "IPA", "POS", "Link_US", "Link_UK",])
for i,m in enumerate(mots):
    try:
        name = m.findAll('span',{"class":"HWD"})[0].text
        pos = m.findAll('span',{"class":"POS"})[0].text
        ipa = m.findAll('span',{"class":"PRON"})[0].text
        audio_UK_url = m.findAll('span',{"class":"speaker brefile fas fa-volume-up hideOnAmp"})[0]["data-src-mp3"]
        audio_US_url = m.findAll('span',{"class":"speaker amefile fas fa-volume-up hideOnAmp"})[0]["data-src-mp3"]
        df.loc[i] = [name, ipa, pos, audio_US_url, audio_UK_url]
    except:
        ok =1
df

,Mot,IPA,POS,Link_US,Link_UK
0,abacus,ˈæbəkəs,noun,https://www.ldoceonline.com/media/english/ameP...,https://www.ldoceonline.com/media/english/breP...


## 4.1. On résume les resultats par site en fonction

In [14]:
def scrap_linguee(word):
    urlTarget = f"https://www.linguee.fr/anglais-francais/traduction/{word}.html"
    response = requests.get(urlTarget)
    love = BeautifulSoup(response.text, 'html.parser')
    body = BeautifulSoup(response.text, 'html.parser').find('body')
    span = body.findAll('span',{"class":"tag_lemma"})
    df = pd.DataFrame(columns=["Mot", "IPA", "POS", "Link_US", "Link_UK",])
    for i,v in enumerate(span):
        name = v.findAll('a',{"class":"dictLink"})
        # l'IPA n'est pas mentionné dans Linguee
        ipa = ""
        link = v.findAll('a',{"class":"audio"})
        pos = v.findAll('span',{"class":"tag_wordtype"})
        if len(pos)>0:
            for l in link:
                audio_US = l['onclick'].split('"')[1]
                audio_UK = l['onclick'].split('"')[5]
                audio_US_url = f"https://www.linguee.fr/mp3/{audio_US}.mp3"
                audio_UK_url = f"https://www.linguee.fr/mp3/{audio_UK}.mp3"
                df.loc[i] = [name[0].text, ipa, pos[0].text, audio_US_url, audio_UK_url]
    return df

def scrap_cambridge(word):
    session = HTMLSession()
    urlTarget = f"https://dictionary.cambridge.org/dictionary/english/{word}"
    response = session.get(urlTarget)
    love = BeautifulSoup(response.text, 'html.parser')
    body = BeautifulSoup(response.text, 'html.parser').find('body')
    mots = body.findAll('div',{"class":"pos-header dpos-h"})
    df = pd.DataFrame(columns=["Mot", "IPA", "POS", "Link_US", "Link_UK",])
    for i,m in enumerate(mots):
        name = m.findAll('span',{"class":"hw dhw"})[0].text
        ipa = m.findAll('span',{"class":"pron dpron"})[0].text.replace("/", "")
        div = m.findAll('div',{"class":"posgram dpos-g hdib lmr-5"})
        for d in div:
            pos = d.text.split(" ")[0]
            try:
                audio_UK = m.findAll('span',{"class":"uk dpron-i"})[0].findAll('source')[0]["src"]
                audio_US = m.findAll('span',{"class":"us dpron-i"})[0].findAll('source')[0]["src"]
                audio_US_url = f"https://dictionary.cambridge.org{audio_US}"
                audio_UK_url = f"https://dictionary.cambridge.org{audio_UK}"
                df.loc[i] = [name, ipa, pos, audio_US_url, audio_UK_url]
            except:
                ok = 1
    return df

def scrap_longman(word):
    session = HTMLSession()
    urlTarget = f"https://www.ldoceonline.com/dictionary/{word}"
    response = session.get(urlTarget)
    love = BeautifulSoup(response.text, 'html.parser')
    body = BeautifulSoup(response.text, 'html.parser').find('body')
    mots = body.findAll('span',{"class":"Head"})
    df = pd.DataFrame(columns=["Mot", "IPA", "POS", "Link_US", "Link_UK",])
    for i,m in enumerate(mots):
        try:
            name = m.findAll('span',{"class":"HWD"})[0].text
            pos = m.findAll('span',{"class":"POS"})[0].text
            ipa = m.findAll('span',{"class":"PRON"})[0].text
            audio_UK_url = m.findAll('span',{"class":"speaker brefile fas fa-volume-up hideOnAmp"})[0]["data-src-mp3"]
            audio_US_url = m.findAll('span',{"class":"speaker amefile fas fa-volume-up hideOnAmp"})[0]["data-src-mp3"]
            df.loc[i] = [name, ipa, pos, audio_US_url, audio_UK_url]
        except:
            ok =1
    return df

def show(word):
    print("Mot :", word.capitalize())

    list_function = [scrap_linguee, scrap_cambridge, scrap_longman]
    
    for f in list_function:
        print("\nSite :", f.__name__.split("_")[1].capitalize())
        
        df = f(word)
    
        out = widgets.Output()
        with out:
            tableau_label = '<table><tr><th style="text-align:center">IPA</th>\
                                        <th style="text-align:center">POS</th>\
                                        <th style="text-align:center">Audio US</th>\
                                        <th style="text-align:center">Audio UK</th>\
                                        </tr>'
            for i in range(len(df)):


                tableau_label += '<tr><td>'+df.IPA[i]+'</td>\
                                    <td>'+df.POS[i]+'</td>\
                                      <td> <audio controls src="'+df.Link_US[i]+'"><code>audio</code></audio></td>\
                                      <td> <audio controls src="'+df.Link_UK[i]+'"><code>audio</code></audio></td>\
                                  </tr>'


            tableau_label += '</table>'
            tableau = display.HTML(tableau_label)
            display.display(tableau)


        display.display(out)

## 4.2. On affiche les résultats

In [15]:
show(word)

Mot : Abacus

Site : Linguee


Output()


Site : Cambridge


Output()


Site : Longman


Output()